In [1]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [2]:
product_type = "ambient"
num_time_periods = 365

In [3]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
#article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]
#article_data = article_data.head(5)

In [4]:

if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [5]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [6]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [7]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [8]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [9]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [10]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

In [11]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                tot_ordering += math.ceil((cum_demand- item_demand[t])/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


45955978.49999991
Ordering 41089466.40000009
Fixing  3950988.4000000064
Clearance  62080.700000000026
Max Order  853443.0000000002


In [12]:
buf_cost = 0
vol = 0
for time in range(len(time_indexes)):
    if time in schedule.keys():
        for item in schedule[time].keys():
            vol += schedule[time][item] * cu_per_tu[item] * volume_per_cu[item]
    
    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if vol > warehouse_volume:
        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost

0

In [13]:

# MOdified_schedule is the schedule obtained after improvement heuristic operates.

# Modified schedule
modified_schedule = {}

# Iterate over each article ID
for article_id, orders in schedule.items():
    modified_orders = {}
    ordered_days = sorted(orders.keys())  # Sort the order days in ascending order
    
    # Iterate over pairs of consecutive days
    for i in range(0, len(ordered_days), 2):
        day1 = ordered_days[i]
        if i + 1 < len(ordered_days):
            day2 = ordered_days[i + 1]
        else:
            # If odd number of days, leave the last day as it is
            modified_orders[day1] = orders[day1]
            break

        # Calculate the duration between the two days
        duration = day2 - day1
        
        # Check if the duration is less than the shelf life
        if duration < shelf_life[article_id]:
            # Merge the orders of both days into the first day
            modified_orders[day1] = orders[day1] + orders[day2]
        else:
            # Place orders on their respective days
            modified_orders[day1] = orders[day1]
            modified_orders[day2] = orders[day2]

    modified_schedule[article_id] = modified_orders

# Print the modified schedule
print(modified_schedule)

{'00027afc': {0: 12, 11: 11, 23: 8, 31: 13, 46: 9, 58: 6, 66: 20, 93: 4, 113: 7, 127: 16, 148: 5, 155: 6, 166: 27, 193: 3, 200: 5, 211: 8, 225: 20, 234: 31, 268: 4, 277: 3, 288: 3, 296: 5, 303: 20, 330: 3, 340: 3, 351: 2, 360: 2}, '00084396': {0: 2, 9: 2, 18: 4, 26: 2, 36: 2, 42: 2, 52: 2, 60: 2, 68: 2, 78: 2, 85: 2, 89: 2, 99: 2, 103: 2, 108: 2, 114: 2, 126: 2, 136: 2, 144: 2, 156: 2, 163: 2, 177: 6, 185: 8, 197: 2, 204: 2, 214: 2, 229: 3, 247: 2, 259: 20, 277: 2, 288: 2, 294: 2, 305: 2, 316: 2, 325: 2, 332: 2, 345: 2, 360: 1}, '0025253f': {0: 7, 4: 13, 16: 10, 25: 12, 36: 8, 43: 15, 53: 12, 60: 10, 65: 18, 74: 22, 91: 15, 100: 7, 108: 7, 113: 13, 122: 10, 130: 21, 144: 11, 151: 9, 157: 11, 165: 8, 172: 12, 179: 15, 190: 13, 200: 14, 212: 8, 218: 14, 226: 19, 239: 19, 246: 23, 261: 17, 274: 9, 283: 12, 297: 6, 306: 5, 311: 10, 319: 6, 353: 3, 360: 1}, '00299fa6': {0: 2, 8: 3, 17: 2, 25: 3, 37: 4, 58: 2, 68: 4, 80: 4, 94: 3, 113: 3, 122: 4, 136: 2, 157: 4, 171: 4, 192: 3, 214: 2, 227: 

Cost Function 

In [14]:
total_variable_cost=0
total_fixed_cost=0
total_clearance_cost=0


#Calculate the Total fixed and total variable cost

for article_id, order in modified_schedule.items():
    for day, num_items in order.items():
        variable_cost= ordering_cost_per_tu[article_id] * num_items
        fixed_cost= ordering_cost_fixed[article_id]
        total_variable_cost += variable_cost
        total_fixed_cost += fixed_cost



#Calculate the Clearence Cost
for article_id, order in modified_schedule.items():
    ordered_days = sorted(order.keys())
    first_day = ordered_days[0]
    second_day = ordered_days[1] if len(ordered_days) > 1 else first_day
    ordered_quantity = order[first_day]
    ordered_quantity_in_CU =ordered_quantity*cu_per_tu[article_id]
    demand_sum = sum(demand[article_id].get(d, 0) for d in range(first_day, second_day))
    clearance_cost = (ordered_quantity_in_CU - demand_sum ) * clearing_cost_per_cu[article_id]
    total_clearance_cost += clearance_cost



#Calculation of Buffer Cost 

# Convert the schedule to a day-by-day view
day_by_day_schedule = {}
for article_id, order in modified_schedule.items():
    for day, quantity in order.items():
        if day not in day_by_day_schedule:
            day_by_day_schedule[day] = {}
        day_by_day_schedule[day][article_id] = quantity

# Convert the schedule values from TU to CU using cu_per_tu conversion factor
converted_schedule = {}
for day, articles in day_by_day_schedule.items():
    converted_articles = {article_id: quantity * cu_per_tu[article_id] * volume_per_cu[article_id] for article_id, quantity in articles.items()}
    converted_schedule[day] = converted_articles

# Create a day-by-day view of the demand and calculate volume
day_by_day_demand = {}
total_volume = 0
for article_id, demand_data in demand.items():
    converted_demand = {day: quantity * volume_per_cu[article_id] for day, quantity in demand_data.items()}
    day_by_day_demand[article_id] = converted_demand
    total_volume += sum(converted_demand.values())

# Calculate buffer cost and total volume for each day
buffer_cost_per_day = 25  # Buffer cost per day per cubic meter volume exceeded
warehouse_volume 
total_buffer_cost = 0

for day, articles in converted_schedule.items():
    # Calculate total volume for the current day
    day_volume = sum(articles.values()) + sum(day_by_day_demand[article_id].get(day, 0) for article_id in day_by_day_demand.keys())

    # Check if the total volume exceeds the warehouse volume
    if day_volume > warehouse_volume:
        volume_exceeded = day_volume - warehouse_volume
        buffer_cost = volume_exceeded * buffer_cost_per_day
        print("Buffer Cost for Day", day, ":", buffer_cost)
        total_buffer_cost += buffer_cost


print("total_variable_cost:", total_variable_cost)
print("total_fixed_cost:", total_variable_cost)
print("total_clearance_cost:", total_clearance_cost) 
print("Total Buffer Cost:", total_buffer_cost)
print("Total_cost:",total_variable_cost + total_variable_cost + total_clearance_cost + total_buffer_cost)

Buffer Cost for Day 0 : 4557.139603775007
total_variable_cost: 41089466.39999992
total_fixed_cost: 41089466.39999992
total_clearance_cost: 38303.70000000006
Total Buffer Cost: 4557.139603775007
Total_cost: 82221793.63960361
